In [ ]:
# 패키지 설치
!pip install mxnet==1.6.0 gluonnlp==0.9.1 sentencepiece==0.1.91 pandas==1.0.5 transformers==2.11.0 pytorch_lightning==0.8.1 onnx onnxruntime

# KoGPT2 Chat Convert-To-ONNX Train





In [ ]:
import torch
import numpy as np
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer
from pytorch_lightning.core.lightning import LightningModule
import onnxruntime

tok_path = get_tokenizer()
model, vocab = get_pytorch_kogpt2_model()
tok = SentencepieceTokenizer(tok_path, num_best=0, alpha=0)

class KoGPT2Chat(LightningModule):
    def __init__(self, hparams, **kwargs):
        super(KoGPT2Chat, self).__init__()
        self.hparams = hparams
        self.tok_path = tok_path
        self.neg = -1e18
        self.kogpt2, self.vocab = model, vocab

        self.loss_function = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs):
        # (batch, seq_len, hiddens)
        output, _ = self.kogpt2(inputs)
        return output

if __name__ == '__main__':
    # init
    U_TKN = '<usr>'
    S_TKN = '<sys>'
    BOS = '<s>'
    EOS = '</s>'
    MASK = '<unused0>'
    SENT = '<unused1>'

    # PATH
    MODEL_ONNX_PATH = "./model_chp/sample.onnx"
    # ONNX Settings
    OPERATOR_EXPORT_TYPE = torch._C._onnx.OperatorExportTypes.ONNX
    OPSET_VERSION = 10

    gpt_chat_model = KoGPT2Chat.load_from_checkpoint("./model_chp/model_last.ckpt")

    sent_tokens = tok("")
    sent='0'
    toked = tok(sent)
    q_tok = tok("")
    a = ''
    a_tok = []

    input_ids = torch.LongTensor([
                        vocab[U_TKN]] + vocab[q_tok] +
                        vocab[EOS, SENT] + vocab[sent_tokens] +
                        vocab[EOS, S_TKN] +
                        vocab[a_tok]).unsqueeze(dim=0)

    # export
    torch.onnx.export(gpt_chat_model,
                      input_ids,
                      MODEL_ONNX_PATH,
                      verbose=True,
                      operator_export_type=OPERATOR_EXPORT_TYPE,
                      input_names=['input_ids'],
                      output_names=['output'],
                      dynamic_axes={
                          'input_ids': {
                              1: "sequence"
                          },
                          'output': {
                              1: "sequence"
                          }
                      },
                      opset_version=OPSET_VERSION)
    print("Export of kogpt2 onnx complete!")

# KoGPT2 Chat Convert-To-ONNX Generate

In [ ]:
import torch
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from kogpt2.utils import get_tokenizer
import onnxruntime
from gluonnlp.data import SentencepieceTokenizer
import numpy as np
import time

print(onnxruntime.get_device())

# 폴더 이동
%cd /content/drive/MyDrive/Colab Notebooks/gpt/KoGPT2-chatbot-master

MODEL_ONNX_PATH = "./model_chp/sample.onnx"
_, vocab = get_pytorch_kogpt2_model()
model = onnxruntime.InferenceSession(MODEL_ONNX_PATH)
tok_path = get_tokenizer()
tok = SentencepieceTokenizer(tok_path, num_best=0, alpha=0)

U_TKN = '<usr>'
S_TKN = '<sys>'
BOS = '<s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
sent_tokens = tok("")

while 1:
  q = input('user > ').strip()
  if q == 'quit':
    break
  # if len(q) > 1000:
  #   print("Too many sentences !")
  #   break
  t = time.time()
  q_tok = tok(q)
  a = ''
  a_tok = []
  while 1:
    input_ids = torch.LongTensor([
        vocab[U_TKN]] + vocab[q_tok] +
        vocab[EOS, SENT] + vocab[sent_tokens] +
        vocab[EOS, S_TKN] +
        vocab[a_tok]).unsqueeze(dim=0)
    pred = model.run(None, {'input_ids': np.array(input_ids)})
    gen = vocab.to_tokens(
        torch.argmax(
            torch.tensor(pred),
            dim=-1).squeeze().numpy().tolist())[-1]
    if gen == EOS:
        break
    a += gen.replace('▁', ' ')
    a_tok = tok(a)
  print("Length of sentences : ", len(q))
  print("Running time : ", time.time() - t)
  print("Simsimi > {}".format(a.strip()))